# ANALYSIS OF LUNG CANCER DATASET AND PREDICTION OF SURVIVAL RATE
Lung cancer is a leading cause of cancer-related deaths worldwide. Early detection and accurate prediction of survival rates can significantly improve patient outcomes. In this analysis, we will explore a lung cancer dataset, perform data preprocessing, and build a predictive model to estimate survival rates.

In [17]:
# suppress warnings
import warnings
warnings.filterwarnings('ignore')

# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

## EXTRACTION
In this step, we will load the lung cancer dataset from a CSV file. The dataset contains various features related to lung cancer patients, including demographic information, clinical data, and survival outcomes.

**Loading the Dataset:**

In [18]:
# type: ignore
# loading the dataset
cancer = pd.read_csv("../data/raw.csv")

# convert the dataset to a dataframe
cancer_df = pd.DataFrame(cancer)
cancer_df.head()


,id,age,gender,country,diagnosis_date,cancer_stage,family_history,smoking_status,bmi,cholesterol_level,hypertension,asthma,cirrhosis,other_cancer,treatment_type,end_treatment_date,survived
0,773685,37.0,Male,Lithuania,2015-09-30,Stage II,No,Current Smoker,34.5,241,0,0,0,0,Surgery,2017-05-16,0
1,278120,63.0,Female,Hungary,2024-04-01,Stage III,No,Passive Smoker,22.2,162,1,1,0,0,Combined,2025-12-10,0
2,810423,63.0,Female,Belgium,2015-05-08,Stage III,No,Former Smoker,22.8,230,0,0,1,0,Combined,2016-11-23,1
3,443588,71.0,Male,Denmark,2014-10-05,Stage II,No,Never Smoked,32.1,293,0,0,0,0,Chemotherapy,2016-06-19,1
4,701479,45.0,Female,Cyprus,2015-07-05,Stage I,No,Current Smoker,29.0,173,1,0,0,0,Surgery,2017-01-31,0


**a) Full extraction**
This step involves loading the entire dataset from a CSV file into a DataFrame. A full extraction is useful when we want to analyze the entire dataset without any filtering or selection creteria.



In [19]:
# full extraction
full_extraction = pd.read_csv("../data/raw.csv")
print(f"Pulled {len(full_extraction)} rows via full extraction.")
full_extraction.head()

Pulled 20000 rows via full extraction.


,id,age,gender,country,diagnosis_date,cancer_stage,family_history,smoking_status,bmi,cholesterol_level,hypertension,asthma,cirrhosis,other_cancer,treatment_type,end_treatment_date,survived
0,773685,37.0,Male,Lithuania,2015-09-30,Stage II,No,Current Smoker,34.5,241,0,0,0,0,Surgery,2017-05-16,0
1,278120,63.0,Female,Hungary,2024-04-01,Stage III,No,Passive Smoker,22.2,162,1,1,0,0,Combined,2025-12-10,0
2,810423,63.0,Female,Belgium,2015-05-08,Stage III,No,Former Smoker,22.8,230,0,0,1,0,Combined,2016-11-23,1
3,443588,71.0,Male,Denmark,2014-10-05,Stage II,No,Never Smoked,32.1,293,0,0,0,0,Chemotherapy,2016-06-19,1
4,701479,45.0,Female,Cyprus,2015-07-05,Stage I,No,Current Smoker,29.0,173,1,0,0,0,Surgery,2017-01-31,0


**b) Incremental extraction** 
This step involves loading only the new or updated data from the source. Incremental extraction is useful when we want to keep our dataset up-to-date without reloading the entire dataset. In this case, we will filter the DataFrame to include only rows where the `year` column is greater than a specified value that is 2020-01-01

In [20]:
# set the last extraction date
last_extraction = ("2020-01-01")

# Load cancer dataset
incremental_ext = pd.read_csv("../data/raw.csv")

# convert diagnosis date to datetime
incremental_ext['diagnosis_date'] = pd.to_datetime(incremental_ext['diagnosis_date'], errors='coerce')

# filter to include only rows where the diagnosis date is greater than the last extraction date
incremental_ext = incremental_ext[incremental_ext["diagnosis_date"] > pd.to_datetime(last_extraction)]

# reset the index of the DataFrame
incremental_ext.reset_index(drop=True, inplace=True)

# print output
print(f"Pulled {len(incremental_ext)} rows via incremental extraction.")
incremental_ext.head()

Pulled 8814 rows via incremental extraction.


,id,age,gender,country,diagnosis_date,cancer_stage,family_history,smoking_status,bmi,cholesterol_level,hypertension,asthma,cirrhosis,other_cancer,treatment_type,end_treatment_date,survived
0,278120,63.0,Female,Hungary,2024-04-01,Stage III,No,Passive Smoker,22.2,162,1,1,0,0,Combined,2025-12-10,0
1,100970,54.0,Female,Croatia,2021-08-14,Stage IV,Yes,Never Smoked,36.2,258,1,1,1,0,Radiation,2022-12-13,0
2,684392,60.0,Male,Latvia,2023-01-22,Stage III,No,Passive Smoker,18.7,195,1,1,0,0,Combined,2024-03-15,0
3,746694,55.0,Female,Hungary,2020-08-04,Stage III,No,Never Smoked,28.8,161,1,1,0,0,Radiation,2021-05-11,0
4,566016,46.0,Male,Spain,2024-01-03,Stage I,Yes,Current Smoker,37.3,257,1,0,0,0,Surgery,2025-04-28,0


## Description of the Dataset
The lung cancer dataset contained 890000 records initially. After performing an incremental extraction, we filtered the dataset to include only records from the year `2023-01-01` onwards. The resulting dataset contained 125,749 records.

The data contains the following columns:
- `patient_id`: Unique identifier for each patient.
- `age`: Age of the patient.
- `gender`: Either male or female.
- `country`: Country of the patient.
- `diagnosis_date`: Date when the patient was diagnosed with lung cancer.
- `cancer_stage`: Stage of lung cancer at the time of diagnosis.
- `family_history`: Indicates whether the patient has a family history of lung cancer.
- `smoking_status`: Indicates whether the patient is a passive smoker, never smoked, or former smoker.
- `BMI`: Body Mass Index of the patient.
- `choleterol`: Cholesterol level of the patient.
- `hypertension`: Indicates whether the patient has hypertension.
- `asthma`: Indicates whether the patient has asthma.
- `cirrhosis`: Indicates whether the patient has cirrhosis.
- `other_cancer`: Indicates whether the patient has other types of cancer.
- `treatment_type`: Type of treatment received by the patient.
- `end_treatment_date`: Date when the treatment ended.
- `survival_status`: Indicates whether the patient survived or not.

### Data Cleaning
- checking for null values
- checking for duplicate values
- checking datatypes

In [21]:
# checking for null values
missing = incremental_ext.isnull().sum()
print(f"Total number of missing values:\n{missing}")

# checking for duplicate values
dups = incremental_ext.duplicated().sum()
print(f"Total number of duplicate values: {dups}")

# checking the datatypes 
print("The datatypes of the columns:\n")
print(incremental_ext.dtypes)

# describing the dataset
incremental_ext.describe()

Total number of missing values:
id                    0
age                   0
gender                0
country               0
diagnosis_date        0
cancer_stage          0
family_history        0
smoking_status        0
bmi                   0
cholesterol_level     0
hypertension          0
asthma                0
cirrhosis             0
other_cancer          0
treatment_type        0
end_treatment_date    0
survived              0
dtype: int64
Total number of duplicate values: 0
The datatypes of the columns:

id                             int64
age                          float64
gender                        object
country                       object
diagnosis_date        datetime64[ns]
cancer_stage                  object
family_history                object
smoking_status                object
bmi                          float64
cholesterol_level              int64
hypertension                   int64
asthma                         int64
cirrhosis                      int64

,id,age,diagnosis_date,bmi,cholesterol_level,hypertension,asthma,cirrhosis,other_cancer,survived
count,8814.000000,8814.000000,8814,8814.00000,8814.000000,8814.000000,8814.00000,8814.000000,8814.000000,8814.000000
mean,444018.895053,55.064783,2022-03-13 09:45:52.076242176,30.46990,232.968459,0.757091,0.46290,0.222941,0.091899,0.219310
min,32.000000,15.000000,2020-01-02 00:00:00,16.00000,150.000000,0.000000,0.00000,0.000000,0.000000,0.000000
25%,221429.500000,48.000000,2021-01-28 06:00:00,23.20000,195.000000,1.000000,0.00000,0.000000,0.000000,0.000000
50%,444029.000000,55.000000,2022-03-17 00:00:00,30.50000,241.000000,1.000000,0.00000,0.000000,0.000000,0.000000
75%,663991.000000,62.000000,2023-04-21 00:00:00,37.80000,271.000000,1.000000,1.00000,0.000000,0.000000,0.000000
max,889707.000000,95.000000,2024-05-30 00:00:00,45.00000,300.000000,1.000000,1.00000,1.000000,1.000000,1.000000
std,256503.081395,9.929469,NaN,8.39229,43.742971,0.428865,0.49865,0.416243,0.288900,0.413803


The extracted dataset lucks missing values and duplicate records, making it suitable for further analysis

# TRANSFORMATION
In this step, we will perform data preprocessing and transformation to prepare the dataset for analysis. This includes the following:

1. **Date Conversion**: Convert the `diagnosis_date` and `end_treatment_date` columns to datetime format. This enable us to perform date-related operations and calculations.

In [22]:
#type: ignore
# converting the diagnosis_date and end_treatment_date columns to datetime format
incremental_ext['diagnosis_date'] = pd.to_datetime(incremental_ext['diagnosis_date'], errors='coerce') 
incremental_ext['end_treatment_date'] = pd.to_datetime(incremental_ext['end_treatment_date'], errors='coerce')
incremental_ext.head()

,id,age,gender,country,diagnosis_date,cancer_stage,family_history,smoking_status,bmi,cholesterol_level,hypertension,asthma,cirrhosis,other_cancer,treatment_type,end_treatment_date,survived
0,278120,63.0,Female,Hungary,2024-04-01,Stage III,No,Passive Smoker,22.2,162,1,1,0,0,Combined,2025-12-10,0
1,100970,54.0,Female,Croatia,2021-08-14,Stage IV,Yes,Never Smoked,36.2,258,1,1,1,0,Radiation,2022-12-13,0
2,684392,60.0,Male,Latvia,2023-01-22,Stage III,No,Passive Smoker,18.7,195,1,1,0,0,Combined,2024-03-15,0
3,746694,55.0,Female,Hungary,2020-08-04,Stage III,No,Never Smoked,28.8,161,1,1,0,0,Radiation,2021-05-11,0
4,566016,46.0,Male,Spain,2024-01-03,Stage I,Yes,Current Smoker,37.3,257,1,0,0,0,Surgery,2025-04-28,0


2) **Feature Engineering**: 

a) Create a new column `treatment_duration` that calculates the duration of treatment in days by subtracting the `diagnosis_date` from the `end_treatment_date`. This will help us understand the impact of treatment duration on survival rates.

In [23]:
incremental_ext['treatment_duration'] = (incremental_ext['end_treatment_date'] - incremental_ext['diagnosis_date']).dt.days
incremental_ext.head()

,id,age,gender,country,diagnosis_date,cancer_stage,family_history,smoking_status,bmi,cholesterol_level,hypertension,asthma,cirrhosis,other_cancer,treatment_type,end_treatment_date,survived,treatment_duration
0,278120,63.0,Female,Hungary,2024-04-01,Stage III,No,Passive Smoker,22.2,162,1,1,0,0,Combined,2025-12-10,0,618
1,100970,54.0,Female,Croatia,2021-08-14,Stage IV,Yes,Never Smoked,36.2,258,1,1,1,0,Radiation,2022-12-13,0,486
2,684392,60.0,Male,Latvia,2023-01-22,Stage III,No,Passive Smoker,18.7,195,1,1,0,0,Combined,2024-03-15,0,418
3,746694,55.0,Female,Hungary,2020-08-04,Stage III,No,Never Smoked,28.8,161,1,1,0,0,Radiation,2021-05-11,0,280
4,566016,46.0,Male,Spain,2024-01-03,Stage I,Yes,Current Smoker,37.3,257,1,0,0,0,Surgery,2025-04-28,0,481


b) Create a new column `comorbidities_count` that counts the number of comorbidities (hypertension, asthma, cirrhosis, and other_cancer) for each patient. Comorbidity is the presence of one or more additional diseases or disorders co-occurring with a primary disease. This will help us understand the impact of comorbidities on survival rates.


In [24]:
# creating a new column
incremental_ext['comorbidities_count'] = incremental_ext[['hypertension', 'asthma', 'cirrhosis', 'other_cancer']].sum(axis=1)
incremental_ext.head()

,id,age,gender,country,diagnosis_date,cancer_stage,family_history,smoking_status,bmi,cholesterol_level,hypertension,asthma,cirrhosis,other_cancer,treatment_type,end_treatment_date,survived,treatment_duration,comorbidities_count
0,278120,63.0,Female,Hungary,2024-04-01,Stage III,No,Passive Smoker,22.2,162,1,1,0,0,Combined,2025-12-10,0,618,2
1,100970,54.0,Female,Croatia,2021-08-14,Stage IV,Yes,Never Smoked,36.2,258,1,1,1,0,Radiation,2022-12-13,0,486,3
2,684392,60.0,Male,Latvia,2023-01-22,Stage III,No,Passive Smoker,18.7,195,1,1,0,0,Combined,2024-03-15,0,418,2
3,746694,55.0,Female,Hungary,2020-08-04,Stage III,No,Never Smoked,28.8,161,1,1,0,0,Radiation,2021-05-11,0,280,2
4,566016,46.0,Male,Spain,2024-01-03,Stage I,Yes,Current Smoker,37.3,257,1,0,0,0,Surgery,2025-04-28,0,481,1


c) Level binning: This will involve creating bins for the `age` column to categorize patients into groups like `children`, `adolescents`, `adults`, and `elderly`. This will help us analyze survival rates based on age groups.

We will also create bins for the `BMI` column to categorize patients into groups like `underweight`, `normal`, `overweight`, and `obese`. This will help us analyze survival rates based on BMI categories.

Finally, we will create bins for the `cholesterol` column to categorize patients into groups like `Desirable`, `Borderline high`, and `High`. This will help us analyze survival rates based on cholesterol levels.

This can simplify relationships, make models more robust to outliers, and allow for easier interpretation of certain patterns, especially for visualization and initial exploration.

In [25]:
# creating age bing
incremental_ext['age_group'] = pd.cut(incremental_ext['age'], bins=[0, 12, 19, 59, 100], labels=['children', 'adolescents', 'adults', 'elderly'], right=False)

# creating bmi bins
incremental_ext['bmi_category'] = pd.cut(incremental_ext['bmi'], bins=[0, 18.5, 24.9, 29.9, 100], labels=['underweight', 'normal', 'overweight', 'obese'], right=False)   

# creating cholestral bins
incremental_ext['cholesterol_category'] = pd.cut(incremental_ext['cholesterol_level'], bins=[0, 200, 239, 1000], labels=['Desirable', 'Borderline high', 'High'], right=False)
incremental_ext.head()

,id,age,gender,country,diagnosis_date,cancer_stage,family_history,smoking_status,bmi,cholesterol_level,...,cirrhosis,other_cancer,treatment_type,end_treatment_date,survived,treatment_duration,comorbidities_count,age_group,bmi_category,cholesterol_category
0,278120,63.0,Female,Hungary,2024-04-01,Stage III,No,Passive Smoker,22.2,162,...,0,0,Combined,2025-12-10,0,618,2,elderly,normal,Desirable
1,100970,54.0,Female,Croatia,2021-08-14,Stage IV,Yes,Never Smoked,36.2,258,...,1,0,Radiation,2022-12-13,0,486,3,adults,obese,High
2,684392,60.0,Male,Latvia,2023-01-22,Stage III,No,Passive Smoker,18.7,195,...,0,0,Combined,2024-03-15,0,418,2,elderly,normal,Desirable
3,746694,55.0,Female,Hungary,2020-08-04,Stage III,No,Never Smoked,28.8,161,...,0,0,Radiation,2021-05-11,0,280,2,adults,overweight,Desirable
4,566016,46.0,Male,Spain,2024-01-03,Stage I,Yes,Current Smoker,37.3,257,...,0,0,Surgery,2025-04-28,0,481,1,adults,obese,High


5) **Creatubg age bins**: Creating age bins to categorize patients into age groups. This will help us analyze survival rates based on age groups.
(Tanveer Section)

In [26]:
if 'age' in incremental_ext.columns:
    bins = [0, 18, 30, 45, 60, 75, incremental_ext['age'].max() + 1] 
    labels = ['<18', '18-29', '30-44', '45-59', '60-74', '75+']
    incremental_ext['age_group'] = pd.cut(incremental_ext['age'], bins=bins, labels=labels, right=False, include_lowest=True)
    print("Added 'age_group' column.")

Added 'age_group' column.


5) **Creatubg Comorbidity Counts**: Creating colimsn that count the number of comorbidities for each patient. This will help us understand the impact of comorbidities on survival rates.(Tanveer Section)

In [27]:
comorbidity_flags = ['hypertension', 'asthma', 'cirrhosis', 'other_cancer']
# Ensure these are 0/1 before summing
existing_comorbidities = [col for col in comorbidity_flags if col in incremental_ext.columns and pd.api.types.is_numeric_dtype(incremental_ext[col])]
if existing_comorbidities:
    incremental_ext['comorbidity_count'] = incremental_ext[existing_comorbidities].sum(axis=1)
    print(f"Added 'comorbidity_count' based on: {existing_comorbidities}.")
else:
    print("No numerical comorbidity flags found to create 'comorbidity_count'.")

Added 'comorbidity_count' based on: ['hypertension', 'asthma', 'cirrhosis', 'other_cancer'].


6) **Creating Diagnosis year/month/Qarter for time series analysis later**: Setting up for Time Series analysis Later.(Tanveer Section)

In [28]:
if 'diagnosis_date' in incremental_ext.columns:
    incremental_ext['diagnosis_year'] = incremental_ext['diagnosis_date'].dt.year
    incremental_ext['diagnosis_month'] = incremental_ext['diagnosis_date'].dt.month
    incremental_ext['diagnosis_quarter'] = incremental_ext['diagnosis_date'].dt.quarter
    incremental_ext['diagnosis_year_month'] = incremental_ext['diagnosis_date'].dt.to_period('M') # For time-series grouping
    print("Added 'diagnosis_year', 'diagnosis_month', 'diagnosis_quarter', 'diagnosis_year_month'.")


Added 'diagnosis_year', 'diagnosis_month', 'diagnosis_quarter', 'diagnosis_year_month'.


In [29]:
# Saving the transformed dataset
incremental_ext.to_csv("../data/transformed_data.csv", index=False)

3) **Categorical Encoding**

a) **One-Hot Encoding**: Convert categorical variables such as `gender`, `country`, `cancer_stage`, `family_history`, `smoking_status`, and `treatment_type` into numerical format where each category becomes a new binary. This is necessary for machine learning algorithms that require numerical input. For encoding categorical variables, we will use one-hot encoding.

In [30]:
# columns to encode
encoded_col = ['gender', 'country', 'smoking_status', 'treatment_type', 'family_history']

# drop first to avoid multicollenearity
df_encoded_pd = pd.get_dummies(incremental_ext, columns=encoded_col, drop_first=True, dtype=int)
print("\nInfo after One-Hot Encoding:")
print(df_encoded_pd.info())
print("DataFrame after One-Hot Encoding with pd.get_dummies():")
df_encoded_pd.head()


Info after One-Hot Encoding:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8814 entries, 0 to 8813
Data columns (total 56 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   id                             8814 non-null   int64         
 1   age                            8814 non-null   float64       
 2   diagnosis_date                 8814 non-null   datetime64[ns]
 3   cancer_stage                   8814 non-null   object        
 4   bmi                            8814 non-null   float64       
 5   cholesterol_level              8814 non-null   int64         
 6   hypertension                   8814 non-null   int64         
 7   asthma                         8814 non-null   int64         
 8   cirrhosis                      8814 non-null   int64         
 9   other_cancer                   8814 non-null   int64         
 10  end_treatment_date             8814 non-null   datetim

,id,age,diagnosis_date,cancer_stage,bmi,cholesterol_level,hypertension,asthma,cirrhosis,other_cancer,...,country_Slovenia,country_Spain,country_Sweden,smoking_status_Former Smoker,smoking_status_Never Smoked,smoking_status_Passive Smoker,treatment_type_Combined,treatment_type_Radiation,treatment_type_Surgery,family_history_Yes
0,278120,63.0,2024-04-01,Stage III,22.2,162,1,1,0,0,...,0,0,0,0,0,1,1,0,0,0
1,100970,54.0,2021-08-14,Stage IV,36.2,258,1,1,1,0,...,0,0,0,0,1,0,0,1,0,1
2,684392,60.0,2023-01-22,Stage III,18.7,195,1,1,0,0,...,0,0,0,0,0,1,1,0,0,0
3,746694,55.0,2020-08-04,Stage III,28.8,161,1,1,0,0,...,0,0,0,0,1,0,0,1,0,0
4,566016,46.0,2024-01-03,Stage I,37.3,257,1,0,0,0,...,0,1,0,0,0,0,0,0,1,1


In [31]:
# Saving the encoded dataset
df_encoded_pd.to_csv("../data/encoded_data.csv", index=False)